# PISM Greenland Tutorial

The flow of glaciers can be described as Stokes flow, obeying conservation equations for *mass*, *linear momentum*, and *energy* (*angular momentum* results in a symmetric stress tensor)

$$
\begin{aligned}
\textrm{mass}:  &\nabla \cdot \mathbf{v} & = & 0 \\
\textrm{momentum}:  & \nabla \left( \eta \nabla \cdot \mathbf{v} \right) - \rho \mathbf{g} & = & 0 \\
\textrm{energ}: & \frac{\mathrm{d} E}{\mathrm{d} t}
\end{aligned}
$$

The visosity of ice, $\eta = A(T) \left(v \right)$. Ice has a memory, which significantly complicates the simulation of glaciers.

We need initial conditions for these conservation equations.

mass: initial ice geometry $\Omega_0$
momentum: 
energy: initial energy field $E_0$
$$
\begin{aligned}
\textrm{mass}:  & \textrm{ice geometry} \Omega_0 \\
\textrm{momentum}: & \textrm{}  \\
\textrm{energy}: & \textrm{energy} E_0
\end{aligned}
$$

In [ ]:
from functools import partial

from pathlib import Path
import matplotlib.pylab as plt
from matplotlib import colors
import numpy as np
import xarray as xr
import cf_xarray.units  # otherwise we get a Parser error
import pint_xarray

from pism_tutorials.download import download_files
from pism_tutorials.plotting import register_colormaps
from pism_tutorials.processing import preprocess_nc

register_colormaps()
xr.set_options(keep_attrs=True)

In [ ]:
# The name of the PISM Cloud S3 bucket
bucket_name = "pism-cloud-data"

In [ ]:
tutorial_files = {"initial_state": {"path": "initial_states", "files": ["g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc"]}, 
                  "basal_heatflux": {"path": "bheatflux", "files": ["geothermal_heat_flow_map_10km.nc"]},
                  "climate": {"path": "ismip6", "files": ["MARv3.9_MIROC5-rcp26_climate_1960-2100_v1.nc", "MARv3.9_MIROC5-rcp85_climate_1960-2100_v1.nc"]},
                  "ocean": {"path": "ismip6", "files": ["MAR3.9_MIROC-ESM-CHEM_rcp26_ocean_1960-2100_v4.nc", "MAR3.9_MIROC-ESM-CHEM_rcp85_ocean_1960-2100_v4.nc"]},
                  "grid": {"path": "grids", "files": ["pism-bedmachine.nc"]}}

to_download = []
for forcing_type, forcing in tutorial_files.items():
    for f in forcing["files"]:
        d = Path(forcing["path"])
        d.mkdir(parents=True, exist_ok=True)
        p = d / Path(f)
        to_download.append(str(p))


In [ ]:
# Download files if not exist.
# Set "overwrite=True" to overwrite existing files.
download_files(bucket_name, to_download, overwrite=False)

In [ ]:
init_file = Path(tutorial_files["initial_state"]["path"]) / Path(tutorial_files["initial_state"]["files"][0])
init_ds = xr.open_dataset(init_file, decode_timedelta=True)

climate_files = Path(tutorial_files["climate"]["path"]).glob("*_climate_*.nc")
climate_ds = xr.open_mfdataset(climate_files,                       
                               preprocess=partial(preprocess_nc, regexp="_(.+?)_", dim="gcm"),
                               chunks="auto",
                               parallel=True,             
                               decode_cf=True,
                               decode_timedelta=True,
                               combine="nested",
                               concat_dim="gcm")

climate_ds["climatic_mass_balance_anomaly"] = climate_ds["climatic_mass_balance_anomaly"].pint.quantify().pint.to("kg m-2 yr-1")

bhf_file = Path(tutorial_files["basal_heatflux"]["path"]) / Path(tutorial_files["basal_heatflux"]["files"][0])
bhf_ds = xr.open_dataset(bhf_file, decode_timedelta=True)

## Initial geometry

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
init_ds["usurf"].plot(ax=axs[0], cmap="bath_topo", vmin=-500, vmax=2000, extend="both")
init_ds["topg"].plot(ax=axs[1], cmap="bath_topo", vmin=-500, vmax=2000, extend="both")
init_ds["thk"].where(init_ds["thk"] > 0).plot(ax=axs[2], cmap="managua", vmax=3000, extend="max")
axs[0].set_title("ice surface elevation")
axs[1].set_title("sublacial topography")
axs[2].set_title("ice thickness")
for ax in axs:
    ax.set_axis_off()

## Climate forcing

In [ ]:
fig = anomaly_2090s["climatic_mass_balance_anomaly"].plot(col="gcm", vmin=-2500, vmax=2500, cmap="RdBu", extend="both", figsize=(8, 4)).fig
for ax in fig.axes[:2]:
    ax.set_axis_off()
fig = anomaly_2090s["ice_surface_temp_anomaly"].plot(col="gcm", cmap="Reds", vmin=0, vmax=20, extend="max", figsize=(8, 4)).fig
for ax in fig.axes[:2]:
    ax.set_axis_off()

In [ ]:
f = "mpirun -np 4 pism -eisII A -grid.Mx 101 -grid.My 101"

In [ ]:
!$f 

In [ ]:
climate_ds.drop_vars(["time_bnds"]).pint.quantify()

In [ ]:
fig.ax